<a href="https://colab.research.google.com/github/AnujM11/Tensorflow/blob/main/nlp%20cat4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import json
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [12]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
urllib.request.urlretrieve(url, 'sarcasm.json')

('sarcasm.json', <http.client.HTTPMessage at 0x7f5e56ca1c60>)

In [13]:
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

In [14]:
sentences = []
labels = []

In [15]:
with open("sarcasm.json", 'r') as f:
        datastore = json.load(f)

for item in datastore:
  sentences.append(item['headline'])
  labels.append(item['is_sarcastic'])

In [16]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [17]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

callbacks = [
EarlyStopping(
            monitor='val_accuracy',
            min_delta=1e-4,
            patience=3,
            verbose=1
        ),
ModelCheckpoint(
            filepath='mymodel.h5',
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            save_weights_only=False,
            verbose=1
        )
]

    # Need this block to get it to work with TensorFlow 2.x
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [24]:
model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True)),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dense(1, activation='softmax')
    ])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

num_epochs = 30


In [25]:
model.fit(training_padded, training_labels, epochs=num_epochs,validation_data=(testing_padded, testing_labels), verbose=1)

Epoch 1/30
623/625 [============================>.] - ETA: 0s - loss: 0.5312 - accuracy: 0.4398
Epoch 1: val_accuracy did not improve from 0.43673
625/625 [==============================] - 34s 42ms/step - loss: 0.5309 - accuracy: 0.4397 - val_loss: 0.4702 - val_accuracy: 0.4367
Epoch 2/30
624/625 [============================>.] - ETA: 0s - loss: 0.3809 - accuracy: 0.4399
Epoch 2: val_accuracy did not improve from 0.43673
625/625 [==============================] - 15s 23ms/step - loss: 0.3808 - accuracy: 0.4397 - val_loss: 0.4155 - val_accuracy: 0.4367
Epoch 3/30
624/625 [============================>.] - ETA: 0s - loss: 0.3626 - accuracy: 0.4398
Epoch 3: val_accuracy did not improve from 0.43673
625/625 [==============================] - 16s 25ms/step - loss: 0.3626 - accuracy: 0.4397 - val_loss: 0.3903 - val_accuracy: 0.4367
Epoch 4/30
625/625 [==============================] - ETA: 0s - loss: 0.3353 - accuracy: 0.4397
Epoch 4: val_accuracy did not improve from 0.43673
625/625 [====